In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["OMP_NUM_THREADS"]="1"
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math
import sys
import glob
import time
import pickle
from tensorflow.keras.utils import to_categorical
exp_name = 'detect_nvin_intsum' #_predv'
# velocity is not included in input
v_in = False
# velocity is not predicted in output
v_out = False
n_cin = 1 + v_in

Instructions for updating:
non-resource variables are not supported in the long term


# DATA

In [2]:

def read_data_n_label(data, label):
    input_data = np.load(data)[:,[0,1,3,2,9]]
    input_data[:,2] = np.log(input_data[:,2])/16.
    input_data = input_data[input_data[:,0] > 0]
    input_data = input_data[input_data[:,0] < 60]
    input_data = input_data[input_data[:,1] > -40]
    input_data = input_data[input_data[:,1] < 40]

    grid_in = np.zeros((1,600,800,n_cin))
    count_in = np.ones((1,600,800,1))

    for inz in input_data:
        pos_x = int(inz[0]//0.1)
        pos_y = int((inz[1] + 40)//0.1)
        grid_in[0, pos_x, pos_y, 0] += inz[2]
        if v_in:
            grid_in[0, pos_x, pos_y, 1] = (grid_in[0, pos_x, pos_y, 1]*(count_in[0, pos_x, pos_y, 0]-1) + inz[3])/count_in[0, pos_x, pos_y, 0]
        count_in[0, pos_x, pos_y, 0] += 1

    with open(label) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    grid_label = np.zeros((1,300,400,7+v_out))
    if len(content) == 0:
        return grid_in, grid_label, np.array([-1,-1,-1,-1])
    if v_out:
        velocity = (input_data[:,3][input_data[:,-1]>=0])
        velocity = velocity.mean() if velocity.size>0 else np.zeros(1)
        velocity = velocity/4
    tar_location = np.fromstring(content[1][1:-1], dtype= float, sep = ' ')
    tar_WL = np.fromstring(content[2][1:-1], dtype= float, sep = ',')
    tar_angle = float(content[3])

    pos0 = int((tar_location[0])//0.2)
    pos1 = int((tar_location[1] + 40)//0.2)

    grid_label[0, pos0, pos1, -1] = 1
    grid_label[0, pos0, pos1, -2] = tar_angle
    grid_label[0, pos0, pos1, 3] = tar_WL[0]
    grid_label[0, pos0, pos1, 4] = tar_WL[1]
 
    
    if tar_WL[0] < tar_WL[1]:
        grid_label[0, pos0, pos1, 3] = tar_WL[1]
        grid_label[0, pos0, pos1, 4] = tar_WL[0]
        grid_label[0, pos0, pos1, -2] = tar_angle + np.pi/2
        
    grid_label[0, pos0, pos1, 1] = tar_location[0] - (pos0*0.2) - 0.1
    grid_label[0, pos0, pos1, 2] = tar_location[1] - (pos1*0.2 -40) -0.1
    if v_out:
        grid_label[0, pos0, pos1, 5] = velocity
    if content[0] == 'bike':
        grid_label[0, pos0, pos1, 0]  = 1
    elif content[0] == 'pedestrian':
        grid_label[0, pos0, pos1, 0]  = 2
    else:
        grid_label[0, pos0, pos1, 0]  = 0

    return grid_in, grid_label, np.array([tar_location[0], tar_location[1], tar_WL[0], tar_WL[1]])

In [3]:
image_in = tf.placeholder(tf.float32, (None,600,800, n_cin))
y_label = tf.placeholder(tf.int32, (None,300, 400 ))
y_label_one_hot = tf.one_hot(y_label, 3, axis=-1)
y_angle = tf.placeholder(tf.float32, (None,300, 400 , 1))
y_wl = tf.placeholder(tf.float32, (None,300, 400 , 2))
y_loc = tf.placeholder(tf.float32, (None,300, 400 , 2))


y_mask = tf.placeholder(tf.float32, (None,300, 400 , 1))

net = tf.layers.conv2d(image_in, 8, 3,strides=2, activation=tf.nn.relu, padding='same')
net = tf.layers.conv2d(net, 24, 3, strides=1,activation=tf.nn.relu, padding='same')
net = tf.layers.conv2d(net, 24, 3, strides=1,activation=tf.nn.relu, padding='same')

x1 = net
net = tf.layers.conv2d(net, 32, 3,strides=2, activation=tf.nn.relu, padding='same')
net = tf.layers.conv2d(net, 64, 3, strides=1,activation=tf.nn.relu, padding='same')
net = tf.layers.conv2d(net, 64, 3, strides=1,activation=tf.nn.relu, padding='same')

x2 = net

up1 = tf.layers.conv2d_transpose(x1, 16, 3, activation=tf.nn.relu, padding='same')
up2 = tf.layers.conv2d_transpose(x2, 32, 3,strides=2, activation=tf.nn.relu, padding='same')

concat = tf.concat([up1, up2], axis = -1)


y_pred_occ = tf.layers.conv2d(concat, 3 , 1, activation=None, padding= 'same')
y_pred_occ_prob = tf.nn.softmax(y_pred_occ, axis = -1)
y_pred_angle = tf.layers.conv2d(concat, 1 , 1, activation=None, padding= 'same')
y_pred_loc   = tf.layers.conv2d(concat, 2 , 1, activation=None, padding= 'same')
y_pred_WL   = tf.layers.conv2d(concat, 2 , 1, activation=tf.nn.softplus, padding= 'same')
log_y_pred_WL = tf.log(y_pred_WL + 0.1)
log_y_wl = tf.log(y_wl + 0.1)

loss_occ = tf.reduce_mean(-y_label_one_hot[:,:, : , 0] *0.008*((1 - y_pred_occ_prob[:,:,:,0])) * tf.log(y_pred_occ_prob[:,:,:,0]  + 0.01 )\
                         -y_label_one_hot[:,:, : , 1] *0.992*((1 - y_pred_occ_prob[:,:,:,1])) * tf.log(y_pred_occ_prob[:,:,:,1] + 0.01)\
                          -y_label_one_hot[:,:, : , 2] *0.992*((1 - y_pred_occ_prob[:,:,:,2])) * tf.log(y_pred_occ_prob[:,:,:,2] + 0.01)\
                         )

loss_angle = tf.reduce_mean(y_mask*tf.abs(tf.sin(y_pred_angle - y_angle)))
loss_loc = tf.reduce_mean(y_mask*tf.abs(y_loc- y_pred_loc))
loss_wl = tf.reduce_mean(y_mask*tf.abs(log_y_wl - log_y_pred_WL))
if v_out:
    y_v = tf.placeholder(tf.float32, (None,300, 400 , 1))
    y_pred_v = tf.layers.conv2d(concat, 1 , 1, activation=None, padding= 'same')
    loss_v = tf.reduce_mean(y_mask*tf.abs(y_v - y_pred_v)*(y_v!=0))
    total_loss = 1.0*loss_occ + 1.25*loss_angle + 0.75*loss_loc + loss_wl + 1 * loss_v
else:
    total_loss = 1.0*loss_occ + 1.25*loss_angle + 0.75*loss_loc + loss_wl



Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.


In [5]:
def dataset(np_seed, seqlen=32):
    Xs = []
    labels = []
    training_folders = glob.glob('training_data/*')
    for j in range(len(training_folders)):
        training_labels = glob.glob(training_folders[j] + '/labels/radar_left_np/*')
        training_X = glob.glob(training_folders[j] + '/radar_left_np/*')
        training_labels.sort()
        training_X.sort()
        Xs.extend(training_X)
        labels.extend(training_labels)
    return Xs, labels

np_seed = 12345
optimizer = tf.train.AdamOptimizer().minimize(total_loss)
init = tf.global_variables_initializer()

saver = tf.train.Saver(max_to_keep=25)
seqlen = 32
with tf.Session() as sess:
    sess.run(init)
    for i_epoch in range(25):
        batch_X = []
        batch_y = []
        t_begin = time.time()
        Xs, labels = dataset(np_seed+i_epoch,seqlen=seqlen)
        labels = np.random.RandomState(np_seed+i_epoch).permutation(labels)
        Xs = np.random.RandomState(np_seed+i_epoch).permutation(Xs)
        n_seq = len(Xs)//seqlen

        for i_seq in range(n_seq):
            for ii in range(seqlen):
                X_in, y_in, _ = read_data_n_label(Xs[i_seq*seqlen+ii], labels[i_seq*seqlen+ii])
                batch_X.append(X_in)
                batch_y.append(y_in)

            batch_X = np.array(batch_X).reshape(32,600,800,n_cin)
            batch_y = np.array(batch_y).reshape(32,300,400,7+v_out)
            if v_out:
                y_label_one_hot_see ,pred_occ, pred_angle, pred_loc, pred_wl, cost,_= sess.run([y_label_one_hot, y_pred_occ,y_pred_angle,y_pred_loc,y_pred_WL, total_loss, optimizer], feed_dict = {image_in: batch_X,\
                                                        y_label: batch_y[:,:,:,0], \
                                                        y_angle: batch_y[:,:,:,-2:-1],\
                                                        y_loc: batch_y[:,:,:,[1,2]], \
                                                        y_wl: batch_y[:,:,:,[3,4]], \
                                                        y_v: batch_y[:,:,:,5:6],\
                                                        y_mask: batch_y[:,:,:,-1:], \
                                                        })
            else:
                y_label_one_hot_see ,pred_occ, pred_angle, pred_loc, pred_wl, cost,_= sess.run([y_label_one_hot, y_pred_occ,y_pred_angle,y_pred_loc,y_pred_WL, total_loss, optimizer], feed_dict = {image_in: batch_X,\
                                                        y_label: batch_y[:,:,:,0], \
                                                        y_angle: batch_y[:,:,:,-2:-1],\
                                                        y_loc: batch_y[:,:,:,[1,2]], \
                                                        y_wl: batch_y[:,:,:,[3,4]], \
                                                        y_mask: batch_y[:,:,:,-1:], \
                                                        })                

            if i_seq % (n_seq//4)==0:
                print('epoch {}'.format(i_epoch))
                print ('cost: ', cost)
                z = pred_occ[-1]
                argmax = z.argmax(axis = 2)
                print ('--------------------------------------------------------')
                print ('Predictions:',np.where(argmax > 0))
                print ('labels:', np.where(batch_y[-1,:,:,-1] > 0.5))
                print ('Pre_ID: ', np.argmax(pred_occ[-1]),', GT_ID: ', np.argmax(batch_y[-1,:,:,-1]))
            batch_X = []
            batch_y = []
        save_path = saver.save(sess, "saved_models/{}/model_e{}.ckpt".format(exp_name, i_epoch))
        t_elapsed = time.time()-t_begin
        print('Time Elapsed: ', t_elapsed)
    save_path = saver.save(sess, "saved_models/{}/model.ckpt".format(exp_name))

epoch 0
cost:  0.0057162866
--------------------------------------------------------
Predictions: (array([46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47,
       47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48,
       48, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49,
       49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50,
       50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 51,
       51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 52, 52, 52,
       52, 52, 52, 52, 52, 52, 52, 52, 53, 53, 53, 53, 53, 53, 53, 53, 53,
       53, 53, 53, 53, 53, 53, 53, 54, 54, 54, 54, 54, 54, 54, 54, 54, 55,
       55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 56, 56, 56,
       56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 57, 57, 57,
       57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 57, 58, 58, 58,
       58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 58, 59, 59, 59, 59,
 

epoch 4
cost:  5.0269928e-06
--------------------------------------------------------
Predictions: (array([34, 34, 57]), array([195, 196, 294]))
labels: (array([34]), array([195]))
Pre_ID:  84852 , GT_ID:  13795
epoch 4
cost:  3.885046e-06
--------------------------------------------------------
Predictions: (array([32, 33, 95, 95, 96, 96]), array([208, 208, 287, 288, 288, 289]))
labels: (array([96]), array([288]))
Pre_ID:  353982 , GT_ID:  38688
epoch 4
cost:  5.8809164e-06
--------------------------------------------------------
Predictions: (array([46, 47, 47]), array([242, 241, 242]))
labels: (array([47]), array([242]))
Pre_ID:  65508 , GT_ID:  19042
epoch 4
cost:  5.0294548e-06
--------------------------------------------------------
Predictions: (array([120, 121]), array([310, 310]))
labels: (array([121]), array([310]))
Pre_ID:  353982 , GT_ID:  48710
Time Elapsed:  386.11036586761475
epoch 5
cost:  6.4170654e-06
--------------------------------------------------------
Prediction

epoch 11
cost:  5.25618e-06
--------------------------------------------------------
Predictions: (array([], dtype=int64), array([], dtype=int64))
labels: (array([], dtype=int64), array([], dtype=int64))
Pre_ID:  352824 , GT_ID:  0
epoch 11
cost:  4.0717614e-06
--------------------------------------------------------
Predictions: (array([45]), array([358]))
labels: (array([45]), array([358]))
Pre_ID:  63462 , GT_ID:  18358
epoch 11
cost:  3.971083e-06
--------------------------------------------------------
Predictions: (array([], dtype=int64), array([], dtype=int64))
labels: (array([], dtype=int64), array([], dtype=int64))
Pre_ID:  72690 , GT_ID:  0
epoch 11
cost:  3.542684e-06
--------------------------------------------------------
Predictions: (array([70, 70, 70, 71, 71]), array([258, 259, 260, 258, 259]))
labels: (array([71]), array([259]))
Pre_ID:  96762 , GT_ID:  28659
Time Elapsed:  385.6167857646942
epoch 12
cost:  5.6460594e-06
------------------------------------------------

epoch 18
cost:  3.2263708e-06
--------------------------------------------------------
Predictions: (array([32, 33]), array([197, 197]))
labels: (array([33]), array([197]))
Pre_ID:  355224 , GT_ID:  13397
epoch 18
cost:  2.832131e-06
--------------------------------------------------------
Predictions: (array([49]), array([215]))
labels: (array([49]), array([215]))
Pre_ID:  352824 , GT_ID:  19815
epoch 18
cost:  3.4685825e-06
--------------------------------------------------------
Predictions: (array([161]), array([290]))
labels: (array([], dtype=int64), array([], dtype=int64))
Pre_ID:  202470 , GT_ID:  0
epoch 18
cost:  2.8624218e-06
--------------------------------------------------------
Predictions: (array([35]), array([211]))
labels: (array([], dtype=int64), array([], dtype=int64))
Pre_ID:  51030 , GT_ID:  0
Time Elapsed:  387.4405038356781
epoch 19
cost:  3.5475275e-06
--------------------------------------------------------
Predictions: (array([119, 120, 120, 120, 121, 121]), a

# Evaluation

In [8]:
y_pred_v= y_pred_angle
for i_epoch in range(12,25):
    saver = tf.train.Saver()
    t_begin = time.time()
    with tf.Session() as sess:
        saver.restore(sess, "saved_models/{}/model_e{}.ckpt".format(exp_name, i_epoch))
        print("Model restored.")
        predictions = {'metadata': [],'label_preds':[], 'scores':[], 'box3d_lidar':[],'velocity':[],'velocity_gt':[]}
        counter = 0
        training_folders = glob.glob('validation_data/*')
        training_folders.sort()
        for folder in range(len(training_folders)):
            training_labels = glob.glob(training_folders[folder] + '/labels/radar_left_np/*')
            print(folder)
            training_X = glob.glob(training_folders[folder] + '/radar_left_np/*')
            training_labels.sort()
            training_X.sort()
            X = []
            y = []
            for i in range(len(training_labels)):
                X_in, y_in, location = read_data_n_label(training_X[i], training_labels[i])
                X.append(X_in)
                y.append(y_in)
            X = np.concatenate(X, axis=0)
            y = np.concatenate(y, axis=0)
            var_names = ['occ','angle','loc','wl', 'v']   
            for i in range(len(X)):
                pred_occ, pred_angle, pred_loc, pred_wl, pred_v= sess.run([ y_pred_occ_prob,y_pred_angle,y_pred_loc,y_pred_WL, y_pred_v], \
                                                                                     feed_dict = {image_in: X[i:(i+1)],})


                pred_loc_idx = np.asarray(np.where(pred_occ[0,:,:,:].argmax(axis = -1)>0)).T
                try:
                    v_gt = y[i][...,5].sum()
                except:
                    v_gt = 0
                predictions['velocity_gt'].append(v_gt*4)
                predictions['metadata'].append(training_X[i])

                predictions['label_preds'].append([])
                predictions['box3d_lidar'].append([])
                predictions['scores'].append([])
                predictions['velocity'].append([])

                for index in range(len(pred_loc_idx)):

                    predictions['label_preds'][counter].append(pred_occ[:,pred_loc_idx[index,0], pred_loc_idx[index,1], [1,2]].argmax(axis = -1)[0])
                    predictions['scores'][counter].append(pred_occ[:,pred_loc_idx[index,0], pred_loc_idx[index,1], [1,2]].max(axis = -1)[0])
                    predictions['velocity'][counter].append(pred_v[:,pred_loc_idx[index,0], pred_loc_idx[index,1], 0]*4)

                    pred_location_x =  pred_loc[:,pred_loc_idx[index,0], pred_loc_idx[index,1],0] + (pred_loc_idx[index,0]*0.2) + 0.1
                    pred_location_y =  pred_loc[:,pred_loc_idx[index,0], pred_loc_idx[index,1],1] + (pred_loc_idx[index,1]*0.2 -40) + 0.1

                    W_location_xy = pred_wl[:,pred_loc_idx[index,0], pred_loc_idx[index,1],0]
                    L_location_xy = pred_wl[:,pred_loc_idx[index,0], pred_loc_idx[index,1],1]

                    angle_xy = pred_angle[:,pred_loc_idx[index,0], pred_loc_idx[index,1],0]
                    predictions['box3d_lidar'][counter].append([])
                    predictions['box3d_lidar'][counter][-1].append(pred_location_x[0])
                    predictions['box3d_lidar'][counter][-1].append(pred_location_y[0])
                    predictions['box3d_lidar'][counter][-1].append(-1)
                    predictions['box3d_lidar'][counter][-1].append(W_location_xy[0])
                    predictions['box3d_lidar'][counter][-1].append(L_location_xy[0])
                    predictions['box3d_lidar'][counter][-1].append(-1)
                    predictions['box3d_lidar'][counter][-1].append(angle_xy[0])
                counter += 1
    fname = "saved_models/{}/model_e{}.pkl".format(exp_name, i_epoch)
    with open(fname, 'wb') as output:
        pickle.dump(predictions, output)
    print(time.time()-t_begin)

INFO:tensorflow:Restoring parameters from saved_models/detect_nvin_intsum/model_e12.ckpt
Model restored.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
43.304078340530396
INFO:tensorflow:Restoring parameters from saved_models/detect_nvin_intsum/model_e13.ckpt
Model restored.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
42.79072594642639
INFO:tensorflow:Restoring parameters from saved_models/detect_nvin_intsum/model_e14.ckpt
Model restored.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
43.005319595336914
INFO:tensorflow:Restoring parameters from saved_models/detect_nvin_intsum/model_e15.ckpt
Model restored.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
43.842657804489136
INFO:tensorflow:Restoring parameters from saved_models/detect_nvin_intsum/model_e16.ckpt
Model restored.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
43.28189516067505
INFO:tensorflow:Restoring parameters from saved_models/detect_nvin_intsum/model_e17.ckpt
Model restored.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
43.05147171020508
INFO:tensorflow:Restoring parameters from saved_m